In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf

import os
import cv2
import gc
from skimage import color, data, restoration
import cv2
import numpy as np
from skimage.restoration import estimate_sigma
from skimage.filters import median
import config
import imutils
import warnings
warnings.filterwarnings('ignore')


/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
def weiner_noise_reduction(img):
    # data.astronaut()
    img = color.rgb2gray(img)
    from scipy.signal import convolve2d
    psf = np.ones((5, 5)) / 25
    img = convolve2d(img, psf, 'same')
    img += 0.1 * img.std() * np.random.standard_normal(img.shape)
    deconvolved_img = restoration.wiener(img, psf, 1100)

    return deconvolved_img



def estimate_noise(img):
    # img = cv2.imread(image_path)
    return estimate_sigma(img, multichannel=True, average_sigmas=True)


def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    enoise = estimate_noise(image)
    noise_free_image = weiner_noise_reduction(image)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    fingerprint = gray - noise_free_image
    fingerprint = fingerprint / 255
    filtered_img = median(fingerprint, selem=None, out=None, mask=None, shift_x=False,
                          shift_y=False, mode='nearest', cval=0.0, behavior='rank')
    colored = cv2.cvtColor(filtered_img, cv2.COLOR_GRAY2BGR)
    # print('-----------------')
    # cv2.imshow('filtered_image', filtered_img)
    # colored = cv2.cvtColor(filtered_img, cv2.COLOR_GRAY2BGR)
    # print(colored)
    # cv2.imshow('colored', colored)
    return colored


In [3]:
import keras
from keras import Model, Sequential, optimizers, applications
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dropout, Dense, Flatten
from keras_applications import resnet50
from keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import config

Using TensorFlow backend.


In [9]:
img_height = 299
img_width = 299

# build the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

In [10]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(2, activation='sigmoid'))

In [11]:
# model.add(top_model) this throws error alternative is below

new_model = Sequential() #new model
for layer in model.layers:
    new_model.add(layer)

new_model.add(top_model) # now this works

In [12]:
# model_aug.load_weights('99 % accurate model.h5')
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
new_model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [13]:
labels_batch

<tf.Tensor 'batch:1' shape=(16, 2) dtype=float32>

In [ ]:

history = new_model.fit(images_batch, labels_batch,
          epochs=1,
          steps_per_epoch=100 // 16, verbose=2)


Instructions for updating:
Use tf.cast instead.
Epoch 1/1


In [ ]:
history.history